In [ ]:
# untuk mengatur timezone
# %pip install pytz

# untuk melakukan geocoding
# %pip install geopy
# %pip install dotenv

%pip install prefect

: 

In [ ]:
# standard library
import pandas as pd
import pandas_gbq as gbq
import numpy as np
import requests

# datetime and timezone library
from datetime import datetime, timezone
from pytz import timezone as tz

# geocoding library
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter

# Google API libraries
from google.oauth2 import service_account

# Logging
import logging
# setup logging
logging.basicConfig(level=logging.INFO)



Table of Content
1. Extract
2. Transform
3. Load

## **Data Extraction**

In [3]:
# Define BASE_URL

# BASE URL for real-time earthquake data (past 24 hours)
BASE_URL = "https://earthquake.usgs.gov/earthquakes/feed/v1.0/summary/all_day.geojson"

# BASE URL for historical earthquake data
BASE_URL_HISTORICAL = "https://earthquake.usgs.gov/fdsnws/event/1/query"

In [4]:
# Get all real-time earthquake data
response = requests.get(BASE_URL)
data = response.json()

In [5]:
# Convert  JSON into Datafram
records = []

for feature in data['features']:
    props = feature['properties']
    coords = feature['geometry']['coordinates']  # [lon, lat, depth]

    record = {
        "id": feature.get("id"),
        "place": props.get("place"),
        "mag": props.get("mag"),
        "time": pd.to_datetime(props.get("time"), unit='ms'),
        "updated": pd.to_datetime(props.get("updated"), unit='ms'),
        "tz": props.get("tz"),
        "felt": props.get("felt"),
        "cdi": props.get("cdi"),
        "mmi": props.get("mmi"),
        "alert": props.get("alert"),
        "status": props.get("status"),
        "tsunami": props.get("tsunami"),
        "sig": props.get("sig"),
        "net": props.get("net"),
        "code": props.get("code"),
        "ids": props.get("ids"),
        "sources": props.get("sources"),
        "types": props.get("types"),
        "longitude": coords[0],
        "latitude": coords[1],
        "depth": coords[2],
        "fetched_at": datetime.now(timezone.utc)
    }
    records.append(record)

records_df = pd.DataFrame(records)

In [6]:
records_df

,id,place,mag,time,updated,tz,felt,cdi,mmi,alert,...,sig,net,code,ids,sources,types,longitude,latitude,depth,fetched_at
0,uw62136047,"22 km ENE of Ashford, Washington",0.36,2025-07-09 03:54:32.100,2025-07-09 03:56:07.120,None,NaN,NaN,NaN,None,...,2,uw,62136047,",uw62136047,",",uw,",",origin,phase-data,",-121.763168,46.849167,-2.110000,2025-07-09 04:00:55.275787+00:00
1,ak0258qa7usu,"93 km NW of Aleneva, Alaska",2.60,2025-07-09 03:54:19.346,2025-07-09 03:55:48.029,None,NaN,NaN,NaN,None,...,104,ak,0258qa7usu,",ak0258qa7usu,",",ak,",",origin,phase-data,",-154.174500,58.575800,13.600000,2025-07-09 04:00:55.276104+00:00
2,tx2025njfuex,"47 km NW of Toyah, Texas",1.40,2025-07-09 03:47:27.056,2025-07-09 03:50:10.979,None,NaN,NaN,NaN,None,...,30,tx,2025njfuex,",tx2025njfuex,",",tx,",",origin,phase-data,",-104.126000,31.635000,5.000000,2025-07-09 04:00:55.276365+00:00
3,ok2025nhpl,"5 km E of Meridian, Oklahoma",1.80,2025-07-09 03:43:04.389,2025-07-09 03:45:58.031,None,NaN,NaN,NaN,None,...,50,ok,2025nhpl,",ok2025nhpl,",",ok,",",origin,phase-data,",-97.189034,35.846992,7.716245,2025-07-09 04:00:55.276647+00:00
4,us7000qbi2,"39 km E of Nemuro, Japan",4.60,2025-07-09 03:39:50.569,2025-07-09 03:56:11.040,None,NaN,NaN,NaN,None,...,326,us,7000qbi2,",us7000qbi2,",",us,",",origin,phase-data,",146.058900,43.347900,119.917000,2025-07-09 04:00:55.276928+00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
453,ci41016631,"0 km E of Loma Linda, CA",0.52,2025-07-08 04:31:21.830,2025-07-08 17:49:28.666,None,NaN,NaN,NaN,None,...,4,ci,41016631,",ci41016631,",",ci,",",nearby-cities,origin,phase-data,scitech-link,",-117.257500,34.048833,15.920000,2025-07-09 04:00:55.356748+00:00
454,av93046044,"57 km WNW of Tyonek, Alaska",-0.90,2025-07-08 04:28:50.720,2025-07-08 21:45:04.050,None,NaN,NaN,NaN,None,...,0,av,93046044,",av93046044,",",av,",",origin,phase-data,",-152.142167,61.249500,-3.140000,2025-07-09 04:00:55.356942+00:00
455,tx2025nhlkqe,"23 km S of Westbrook, Texas",0.90,2025-07-08 04:28:23.863,2025-07-08 23:05:34.736,None,NaN,NaN,NaN,None,...,12,tx,2025nhlkqe,",tx2025nhlkqe,",",tx,",",origin,phase-data,",-101.046000,32.148000,8.642600,2025-07-09 04:00:55.357137+00:00
456,nc75206902,"4 km NNW of The Geysers, CA",1.03,2025-07-08 04:11:39.870,2025-07-08 04:13:13.724,None,NaN,NaN,NaN,None,...,16,nc,75206902,",nc75206902,",",nc,",",nearby-cities,origin,phase-data,",-122.783997,38.803833,3.180000,2025-07-09 04:00:55.357328+00:00


## ***2. Data Transform***


- basic Cleaning
- Data Enrichment

In [7]:
df = records_df.copy()

In [8]:
# Drop duplication records
df = df.drop_duplicates(subset=['id'])

# Drop rows with null magnitude or lacation
df = df.dropna(subset=['mag', 'place','latitude', 'longitude','time'])
df

,id,place,mag,time,updated,tz,felt,cdi,mmi,alert,...,sig,net,code,ids,sources,types,longitude,latitude,depth,fetched_at
0,uw62136047,"22 km ENE of Ashford, Washington",0.36,2025-07-09 03:54:32.100,2025-07-09 03:56:07.120,None,NaN,NaN,NaN,None,...,2,uw,62136047,",uw62136047,",",uw,",",origin,phase-data,",-121.763168,46.849167,-2.110000,2025-07-09 04:00:55.275787+00:00
1,ak0258qa7usu,"93 km NW of Aleneva, Alaska",2.60,2025-07-09 03:54:19.346,2025-07-09 03:55:48.029,None,NaN,NaN,NaN,None,...,104,ak,0258qa7usu,",ak0258qa7usu,",",ak,",",origin,phase-data,",-154.174500,58.575800,13.600000,2025-07-09 04:00:55.276104+00:00
2,tx2025njfuex,"47 km NW of Toyah, Texas",1.40,2025-07-09 03:47:27.056,2025-07-09 03:50:10.979,None,NaN,NaN,NaN,None,...,30,tx,2025njfuex,",tx2025njfuex,",",tx,",",origin,phase-data,",-104.126000,31.635000,5.000000,2025-07-09 04:00:55.276365+00:00
3,ok2025nhpl,"5 km E of Meridian, Oklahoma",1.80,2025-07-09 03:43:04.389,2025-07-09 03:45:58.031,None,NaN,NaN,NaN,None,...,50,ok,2025nhpl,",ok2025nhpl,",",ok,",",origin,phase-data,",-97.189034,35.846992,7.716245,2025-07-09 04:00:55.276647+00:00
4,us7000qbi2,"39 km E of Nemuro, Japan",4.60,2025-07-09 03:39:50.569,2025-07-09 03:56:11.040,None,NaN,NaN,NaN,None,...,326,us,7000qbi2,",us7000qbi2,",",us,",",origin,phase-data,",146.058900,43.347900,119.917000,2025-07-09 04:00:55.276928+00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
453,ci41016631,"0 km E of Loma Linda, CA",0.52,2025-07-08 04:31:21.830,2025-07-08 17:49:28.666,None,NaN,NaN,NaN,None,...,4,ci,41016631,",ci41016631,",",ci,",",nearby-cities,origin,phase-data,scitech-link,",-117.257500,34.048833,15.920000,2025-07-09 04:00:55.356748+00:00
454,av93046044,"57 km WNW of Tyonek, Alaska",-0.90,2025-07-08 04:28:50.720,2025-07-08 21:45:04.050,None,NaN,NaN,NaN,None,...,0,av,93046044,",av93046044,",",av,",",origin,phase-data,",-152.142167,61.249500,-3.140000,2025-07-09 04:00:55.356942+00:00
455,tx2025nhlkqe,"23 km S of Westbrook, Texas",0.90,2025-07-08 04:28:23.863,2025-07-08 23:05:34.736,None,NaN,NaN,NaN,None,...,12,tx,2025nhlkqe,",tx2025nhlkqe,",",tx,",",origin,phase-data,",-101.046000,32.148000,8.642600,2025-07-09 04:00:55.357137+00:00
456,nc75206902,"4 km NNW of The Geysers, CA",1.03,2025-07-08 04:11:39.870,2025-07-08 04:13:13.724,None,NaN,NaN,NaN,None,...,16,nc,75206902,",nc75206902,",",nc,",",nearby-cities,origin,phase-data,",-122.783997,38.803833,3.180000,2025-07-09 04:00:55.357328+00:00


In [9]:
# Convert time columns to datetime if not already
# Convert UTC time columns to datetime with Asia/Jakarta timezone
WIB = tz('Asia/Jakarta')
df['time'] = pd.to_datetime(df['time'], errors='coerce').dt.tz_localize('UTC').dt.tz_convert(WIB)
df['updated'] = pd.to_datetime(df['updated'], errors='coerce').dt.tz_localize('UTC').dt.tz_convert(WIB)
df['fetched_at'] = pd.to_datetime(df['fetched_at'], errors='coerce').dt.tz_convert('UTC').dt.tz_convert(WIB)

In [10]:
#  Filter only valid magnitudes
df = df[df['mag'] >= 0]

# Select relevant columns
selected_columns = [
    "id", "place", "mag", "time", "updated",
    "latitude", "longitude", "depth",
    "tsunami", "sig", "status", "alert", "types",
    "felt", "cdi", "mmi", "fetched_at"
]
df = df[selected_columns]

In [11]:
# 2b. Data Enrichment

# Add Magnitude Category
def classify_magnitude(mag):
        if mag < 2.0:
            return "Micro"
        elif mag < 4.0:
            return "Minor"
        elif mag < 5.5:
            return "Light"
        elif mag < 7.0:
            return "Moderate"
        elif mag < 8.0:
            return "Strong"
        else:
            return "Major"

df['mag_category'] = df['mag'].apply(classify_magnitude)

In [16]:
# Add address details
def get_address_detail(latitude, longitude):
    result = {}
    coder = Nominatim(user_agent="myGeocoder", timeout=3)
    # print(coder.reverse(f"{latitude}, {longitude}"))
    address = coder.reverse(f"{latitude}, {longitude}")

    result["city"] = address.raw.get("address").get("city") if address else np.nan
    result["state"] = address.raw.get("address").get("state") if address else np.nan
    result["country"] = address.raw.get("address").get("country") if address else np.nan
    return result

address_detail = df.apply(lambda row: get_address_detail(row['latitude'], row['longitude']), axis=1, result_type='expand')
df = pd.concat([df, address_detail], axis=1)

In [17]:
# Add time-based columns
df['day_of_week'] = df['time'].dt.day_name()
df['hour_of_day'] = df['time'].dt.hour

In [ ]:
df.to_csv(f'../data/earthquake-{datetime.now().strftime("%Y-%m-%d")}.csv', index=False)

In [23]:
# Bigquery Configuration
PROJECT_ID = "earthquake-jcds0612-465304"
TABLE_ID = "earthquake.events" # Dataset.id
CREDENTIALS_PATH = "../config/earthquake-jcds0612-465304-09269c9802c2.json"

In [21]:
credential = service_account.Credentials.from_service_account_file(CREDENTIALS_PATH)

In [ ]:
gbq.to_gbq(
    dataframe=df,
    destination_table=TABLE_ID,
    project_id=PROJECT_ID, 
    credentials=credential, 
    if_exists='replace')

428 out of 428 rows loaded.or:
